In [12]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [13]:
df = pd.read_csv("/content/car_price_prediction_.csv")

In [14]:
df.head()

,Car ID,Brand,Year,Engine Size,Fuel Type,Transmission,Mileage,Condition,Price,Model
0,1,Tesla,2016,2.3,Petrol,Manual,114832,New,26613.92,Model X
1,2,BMW,2018,4.4,Electric,Manual,143190,Used,14679.61,5 Series
2,3,Audi,2013,4.5,Electric,Manual,181601,New,44402.61,A4
3,4,Tesla,2011,4.1,Diesel,Automatic,68682,New,86374.33,Model Y
4,5,Ford,2009,2.6,Diesel,Manual,223009,Like New,73577.10,Mustang


In [15]:
df = df[["Brand", "Engine Size", "Fuel Type", "Mileage", "Model"]].dropna()

In [16]:
le = LabelEncoder()
df["Brand"] = le.fit_transform(df["Brand"])
df["Fuel Type"] = le.fit_transform(df["Fuel Type"])
df["Model"] = le.fit_transform(df["Model"])

In [17]:
np.random.seed(42)
mask = np.random.rand(len(df)) < 0.5
df.loc[mask, "Engine Size"] = -1

In [18]:
x = df.drop("Engine Size", axis=1)
y = df["Engine Size"]

In [19]:
base_model = LinearRegression()

In [20]:
self_training_model = SelfTrainingClassifier(base_model, criterion='k_best')

In [22]:
# Assuming you want to impute the -1 values in 'Engine Size'
# Separate data with known 'Engine Size' and data with unknown 'Engine Size' (-1)
df_known = df[df["Engine Size"] != -1].copy()
df_unknown = df[df["Engine Size"] == -1].copy()

# Define features (x) and target (y) for the known data
x_known = df_known.drop("Engine Size", axis=1)
y_known = df_known["Engine Size"]

# Define features (x) for the unknown data
x_unknown = df_unknown.drop("Engine Size", axis=1)

# Train a Linear Regression model on the known data
imputation_model = LinearRegression()
imputation_model.fit(x_known, y_known)

# Predict the missing 'Engine Size' values using the trained model
predicted_engine_size = imputation_model.predict(x_unknown)

# Replace the -1 values in the original DataFrame with the predicted values
df.loc[df["Engine Size"] == -1, "Engine Size"] = predicted_engine_size

# Now df has the imputed 'Engine Size' values
print(df.head(20))

    Brand  Engine Size  Fuel Type  Mileage  Model
0       5     3.535111          3   114832     19
1       1     4.400000          1   143190      1
2       0     4.500000          1   181601      3
3       5     4.100000          0    68682     20
4       2     3.396223          0   223009     21
5       0     3.364584          0   246553     24
6       0     3.416941          1   135486     23
7       5     5.300000          2    83030     20
8       3     5.700000          1   120360      8
9       2     1.500000          1   135009     11
10      5     3.415523          0   298875     17
11      2     5.700000          1   169737     21
12      2     4.700000          3   114360     12
13      0     3.384448          1   263894     24
14      1     3.447948          1    65018     26
15      2     3.415947          1   240904     21
16      4     3.467807          1   136817     15
17      0     4.400000          2   192803      2
18      3     3.467034          1    86984      8
